## Exercise 1 - Time Until 5 pm tomorrow
### 1 - Corner cases when calculating time to 5PM tomorrow
 - Tomorrow is unknown (Feb 28 on an unknown year)
 - DST forward, DST backward, start or land mid-fold
 - Run program at midnight, date changes mid-execution
 - Tomorrow is calculated correctly (skipping over a 23 hour day)
 
 ### 2 - The `time_till_5pm_tomorrow` function

In [1]:
from datetime import datetime, timedelta, time, timezone

In [2]:
def time_till_5pm_tomorrow(now=None):
    if now is None:
        now = datetime.now()
    tomorrow = now + timedelta(days=1)
    fivepm = time(hour=17, tzinfo = now.tzinfo)
    tomorrow_fivepm = datetime.combine(tomorrow, fivepm)
    seconds_till = tomorrow_fivepm.timestamp() - now.timestamp()
    return seconds_till/3600

### 3 - Testing `time_till_5pm_tomorrow`

In [3]:
# baseline test cases, run in US Eastern Time
print("expected anwser: unknown")
print(time_till_5pm_tomorrow())

print("expected anwser: 24.0")
print(time_till_5pm_tomorrow(datetime(2024,6,18,17)))

expected anwser: unknown
25.53695006642077
expected anwser: 24.0
24.0


In [4]:
# DST test cases, run in US Eastern Time

# March 9-10, the start of DST
print("expected anwser: 23.0")
print(time_till_5pm_tomorrow(datetime(2024,3,9,17)))

# November 2-3, the end of DST
print("expected anwser: 25.0")
print(time_till_5pm_tomorrow(datetime(2024,11,2,17)))

# March 9 11:30pm, 30 minutes before DST
print("expected anwser: 16.5")
print(time_till_5pm_tomorrow(datetime(2024,3,9,23,30)))


expected anwser: 23.0
23.0
expected anwser: 25.0
25.0
expected anwser: 16.5
16.5


### 4 - Using timezone libraries
I'm using each libraries closest approximation of NYC time, and running the last four tests which should get answers: 24, 23, 25, and 16.5

In [5]:
import pytz, dateutil
from zoneinfo import ZoneInfo

In [6]:
# datetime.timezone on EST (UTC-5:00)
NYC = timezone(-timedelta(hours=5), name="America/New_York")
print(time_till_5pm_tomorrow(datetime(2024,6, 18,17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,11,2, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 23, 30,tzinfo = NYC)))

24.0
24.0
24.0
17.5


In [7]:
# pytz.timezone('America/New_York') run with tzinfo parameter
NYC = pytz.timezone('America/New_York')
print(time_till_5pm_tomorrow(datetime(2024,6, 18,17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,11,2, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 23, 30,tzinfo = NYC)))

24.0
24.0
24.0
17.5


In [8]:
# pytz.timezone('America/New_York') run with normalize
NYC = pytz.timezone('America/New_York')
print(time_till_5pm_tomorrow(NYC.normalize(datetime(2024,6, 18,17,    tzinfo = NYC))))
print(time_till_5pm_tomorrow(NYC.normalize(datetime(2024,3, 9, 17,    tzinfo = NYC))))
print(time_till_5pm_tomorrow(NYC.normalize(datetime(2024,11,2, 17,    tzinfo = NYC))))
print(time_till_5pm_tomorrow(NYC.normalize(datetime(2024,3, 9, 23, 30,tzinfo = NYC))))

23.066666666666666
24.066666666666666
23.066666666666666
17.566666666666666


Because pytz does things differently than the other packages, we get really weird results here. We would need to modify the function to use normalize and localize calls.

In [9]:
# dateutil.tz.gettz('America/New_York')
NYC = dateutil.tz.gettz('America/New_York')
print(time_till_5pm_tomorrow(datetime(2024,6, 18,17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,11,2, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 23, 30,tzinfo = NYC)))

24.0
23.0
25.0
16.5


In [10]:
# zoneinfo.ZoneInfo('America/New_York')
NYC = ZoneInfo('America/New_York')
print(time_till_5pm_tomorrow(datetime(2024,6, 18,17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,11,2, 17,    tzinfo = NYC)))
print(time_till_5pm_tomorrow(datetime(2024,3, 9, 23, 30,tzinfo = NYC)))

24.0
23.0
25.0
16.5


### 4.5 - Conclusions
The only two that work "out of the box" are dateutil.tz and ZoneInfo. datetime.timezone doesn't support DST at all and pytz requires some major changes to the fucnction structure for it to work correctly.

### 5 - Comparing pytz and dateutil.tz
This [ganssle blog](https://blog.ganssle.io/articles/2018/03/pytz-fastest-footgun.html) summarizes it pretty well. pytz attempts to resolve to a single timezone whenever possible and so needs to be "prompted" with normalize and localize to make dynamic changes like DST. dateutil.tz on the other hand, is lazy evaluated and so can check what date it

### 6 - Trying out different libraries
I'm using the same tests with local time, zoneinfo and dateutil.tz. If there's another built in way to handle time zones in the package, I'll also test that.

### 6.1 - Arrow
Unlike the main library, it assumes that unspecified time zones are UTC. The built in time zones work on these tests, and it also interfaces correctly with the other two timezone packages. Stuff like shift and floor are handy and it looks like theres a bunch of other useful methods.

In [11]:
import arrow
def time_till_5pm_tomorrow(now=None):
    if now is None:
        now = arrow.now()
    tomorrow = now.shift(days = 1)
    tomorrow_fivepm = tomorrow.replace(hour=17).floor('hour')
    seconds_till = tomorrow_fivepm.timestamp() - now.timestamp()
    return seconds_till/3600

In [12]:
print(time_till_5pm_tomorrow())
print(time_till_5pm_tomorrow(arrow.Arrow(2024,6 ,18,17)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,17)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,11,2 ,17)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,23,30)))
print("-----")
NYC = 'America/New_York'
print(time_till_5pm_tomorrow(arrow.Arrow(2024,6 ,18,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,11,2 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,23,30,tzinfo=NYC)))
print("-----")
NYC = dateutil.tz.gettz('America/New_York')
print(time_till_5pm_tomorrow(arrow.Arrow(2024,6 ,18,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,11,2 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,23,30,tzinfo=NYC)))
print("-----")
NYC = ZoneInfo('America/New_York')
print(time_till_5pm_tomorrow(arrow.Arrow(2024,6 ,18,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,11,2 ,17,   tzinfo=NYC)))
print(time_till_5pm_tomorrow(arrow.Arrow(2024,3 ,9 ,23,30,tzinfo=NYC)))

25.536928602788183
24.0
24.0
24.0
17.5
-----
24.0
23.0
25.0
16.5
-----
24.0
23.0
25.0
16.5
-----
24.0
23.0
25.0
16.5


### 6.2 - pendulum
Also defaults to UTC (but supports naives if you need them). Similarly helpful stuff to arrow. Doesn't play well with dateutil.tz for some reason. 

In [13]:
import pendulum
def time_till_5pm_tomorrow(now=None):
    if now is None:
        now = pendulum.now()
    tomorrow = now.add(days = 1)
    tomorrow_fivepm = tomorrow.replace(hour=17).start_of('hour')
    seconds_till = tomorrow_fivepm.timestamp() - now.timestamp()
    return seconds_till/3600

In [14]:
print(time_till_5pm_tomorrow())
print(time_till_5pm_tomorrow(pendulum.datetime(2024,6 ,18,17)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,17)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,11,2 ,17)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,23,30)))
print("-----")
NYC = 'America/New_York'
print(time_till_5pm_tomorrow(pendulum.datetime(2024,6 ,18,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,11,2 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,23,30,tz=NYC)))
print("-----")
NYC = dateutil.tz.gettz('America/New_York')
print(time_till_5pm_tomorrow(pendulum.datetime(2024,6 ,18,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,11,2 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,23,30,tz=NYC)))
print("-----")
NYC = ZoneInfo('America/New_York')
print(time_till_5pm_tomorrow(pendulum.datetime(2024,6 ,18,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,11,2 ,17,   tz=NYC)))
print(time_till_5pm_tomorrow(pendulum.datetime(2024,3 ,9 ,23,30,tz=NYC)))

25.536920454700788
24.0
24.0
24.0
17.5
-----
24.0
23.0
25.0
16.5
-----
24.0
24.0
24.0
17.5
-----
24.0
23.0
25.0
16.5


### 6.3 - whenever
I'm a huge fan of type safety (which is why I'm working with python less and less these days), so I like the philosophy. It feels like because the main focus is safety, it has less useful methods than the other two like `humanize` or `floor`. It doesn't interface with any other timezone package, but that's because it's rebuilding the system from the ground up, which is probably a good thing.

In [15]:
import whenever
def time_till_5pm_tomorrow(now=None):
    if now is None:
        now = whenever.UTCDateTime.now()
    tomorrow = now + whenever.days(1)
    tomorrow_fivepm = tomorrow.replace(hour=17, minute=0, second=0, microsecond=0)
    diff = tomorrow_fivepm - now
    return diff.in_hours()

In [16]:
print(time_till_5pm_tomorrow())
print(time_till_5pm_tomorrow(whenever.ZonedDateTime(2024,6 ,18,17,   tz = 'America/New_York')))
print(time_till_5pm_tomorrow(whenever.ZonedDateTime(2024,3 ,9 ,17,   tz = 'America/New_York')))
print(time_till_5pm_tomorrow(whenever.ZonedDateTime(2024,11,2 ,17,   tz = 'America/New_York')))
print(time_till_5pm_tomorrow(whenever.ZonedDateTime(2024,3 ,9 ,23,30,tz = 'America/New_York')))

21.536917499166666
24.0
23.0
25.0
16.5
